# Import Modules

In [1]:
import torch
import torch.nn as nn

# VGG16

In [2]:
VGG16 = [64, 64, "MaxPool", 128, 128, "MaxPool", 256, 256, 256, "MaxPool", 512, 512, 512, "MaxPool", 512, 512, 512, "MaxPool"]

class VGG(nn.Module):
    def __init__(self, input=3, output_shape=1000):
        super(VGG, self).__init__()
        self.input = input
        self.conv_layers = self.create_conv_layers(VGG16)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512*7*7, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, output_shape)
        )
    
    def forward(self, out):
        out = self.conv_layers(out)
        out = self.classifier(out)
        return out
    
    def create_conv_layers(self, architecture):
        layers = []
        input = self.input

        for k in architecture:
            if type(k) == int:
                output = k

                layers += [nn.Conv2d(input, output, 
                                     kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                                     nn.ReLU()]
                input = k
            elif k == "MaxPool":
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        return nn.Sequential(*layers)



# ResNet50

In [8]:
class Block(nn.Module):
    def __init__(self, input, output, identity_downsample=None, stride=(1,1)):
        super(Block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(input, output, kernel_size=(1,1), stride=(1,1), padding=0)
        self.bn1 = nn.BatchNorm2d(output)
        self.conv2 = nn.Conv2d(output, output, kernel_size=(3,3), stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(output)
        self.conv3 = nn.Conv2d(output, output*self.expansion, 
                               kernel_size=(1,1), stride=(1,1), padding=0)
        self.bn3 = nn.BatchNorm2d(output*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
    
    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x = x + identity
        x = self.relu(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self, block, layers, input_shape, output_shape):
        super(ResNet, self).__init__()
        self.input = 64
        self.expansion = block(1,1).expansion
        self.conv1 = nn.Conv2d(input_shape, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2), padding=(1,1))

        ###ResNet Layers
        self.layer1 = self.make_layer(block,layers[0], output=64, stride=(1,1))
        self.layer2 = self.make_layer(block,layers[1], output=128, stride=(1,1))
        self.layer3 = self.make_layer(block,layers[2], output=256, stride=(1,1))
        self.layer4 = self.make_layer(block,layers[3], output=512, stride=(1,1))

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Sequential( 
            nn.Flatten(),
            nn.Linear(512*self.expansion, output_shape)
        )
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.fc(x)
        return x
    
    def make_layer(self, block, num_residual_MyBlocks, output, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.input != output*self.expansion:
            identity_downsample = nn.Sequential(nn.Conv2d(
                self.input, output * self.expansion, kernel_size=(1,1), stride=stride),
                nn.BatchNorm2d(output*4)
            )
        layers.append(block(self.input, output, identity_downsample, stride))
        self.input = output*self.expansion

        for i in range(num_residual_MyBlocks-1):
            layers.append(block(self.input, output))
        
        return nn.Sequential(*layers)


seed = 25
torch.manual_seed(25)
model2 = ResNet(Block, [2,2,2,2], input_shape=1, output_shape=17)
print(f"{model2}")


# Count the total number of parameters
total_params = sum(p.numel() for p in model2.parameters())
print(f"Total number of parameters: {total_params}")


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (identity_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum

# ResNet18

In [9]:
class Block(nn.Module):
    def __init__(self, input, output, identity_downsample=None, stride=(1,1)):
        super(Block, self).__init__()
        self.expansion = 2
        self.conv1 = nn.Conv2d(input, output, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.bn1 = nn.BatchNorm2d(output)
        self.conv2 = nn.Conv2d(output,output*self.expansion, kernel_size=(3,3), stride=stride, padding=(1,1))
        self.bn2 = nn.BatchNorm2d(output*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
    
    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x = x + identity
        x = self.relu(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self, block, layers, input_shape, output_shape):
        super(ResNet, self).__init__()
        self.input = 16
        self.expansion = block(1, 1).expansion # Create an instance to get the expansion attribute
        self.conv1 = nn.Conv2d(input_shape, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.layer1 = self.create_layers(block, layers[0], output=16, stride=1)
        self.layer2 = self.create_layers(block, layers[1], output=32, stride=2)
        self.layer3 = self.create_layers(block, layers[2], output=64, stride=2)
        self.layer4 = self.create_layers(block, layers[3], output=128, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential( 
            nn.Flatten(),
            nn.Linear(128 * self.expansion, output_shape)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.fc(x)
        return x
    
    def create_layers(self, block, num_residual_blocks, output, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.input != output * self.expansion:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.input, output * self.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(output * self.expansion)
            )

        layers.append(block(self.input, output, identity_downsample, stride))
        self.input = output * self.expansion

        for _ in range(num_residual_blocks - 1):
            layers.append(block(self.input, output))
        
        return nn.Sequential(*layers)

seed = 25
torch.manual_seed(25)
model2 = ResNet(Block, [2,2,2,2], input_shape=1, output_shape=17)
print(f"{model2}")


# Count the total number of parameters
total_params = sum(p.numel() for p in model2.parameters())
print(f"Total number of parameters: {total_params}")


ResNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (identity_downsample): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Block(
      (conv1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     

In [ ]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, input, output, identity_downsample=None, stride=1):
        super(Block, self).__init__()
        self.expansion = 2
        self.conv1 = nn.Conv2d(input, output, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(output)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(output, output * self.expansion, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(output * self.expansion)
        self.identity_downsample = identity_downsample
    
    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        
        out += identity
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, layers, input_shape, output_shape):
        super(ResNet, self).__init__()
        self.input = 16
        self.expansion = block(1, 1).expansion  # Create an instance to get the expansion attribute
        self.conv1 = nn.Conv2d(input_shape, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.layer1 = self.create_layers(block, layers[0], output=16, stride=1)
        self.layer2 = self.create_layers(block, layers[1], output=32, stride=2)
        self.layer3 = self.create_layers(block, layers[2], output=64, stride=2)
        self.layer4 = self.create_layers(block, layers[3], output=128, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential( 
            nn.Flatten(),
            nn.Linear(128 * self.expansion, output_shape)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.fc(x)
        return x
    
    def create_layers(self, block, num_residual_blocks, output, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.input != output * self.expansion:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.input, output * self.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(output * self.expansion)
            )

        layers.append(block(self.input, output, identity_downsample, stride))
        self.input = output * self.expansion

        for _ in range(num_residual_blocks - 1):
            layers.append(block(self.input, output))
        
        return nn.Sequential(*layers)

def ResNet18(img_channels=1, output_shape=10):
    return ResNet(Block, [2, 2, 2, 2], img_channels, output_shape)

# Example usage:
model = ResNet18()
print(model)

ResNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (identity_downsample): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Block(
      (conv1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     